In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from tqdm.notebook import tqdm
import math
os.chdir('/gdrive/My Drive/NLP_Project_Programs/Paper_Implementation')
os.listdir()

['Vector Representation of Words',
 'lexicons',
 'Evaluation_Benchmarks',
 'retrofit.ipynb',
 '__pycache__',
 'retrofit.py',
 'Retrofitted_WordVecs.ipynb',
 'toefl_2.ipynb',
 'Check Retrofitted Google Vecs.ipynb',
 'Retrofitted_WordVecs Google_Debug.ipynb',
 'Skip_Gram_Retrofit+All_Benchmark.ipynb',
 'SynRelns.ipynb',
 'Word Similarity Tasks.ipynb',
 'Sentiment Analysis.ipynb',
 'toefl.ipynb']

In [0]:
path = 'Evaluation_Benchmarks/Synonym_Selection_TOEFL/toefl.txt'  # path to TOEFL evaluation dataset file
obj = open(path,'r')
list_questions = []
list_choices = []
list_true_ind = []


for i,line in enumerate(obj):
  line = line.strip().lower().split()
  list_questions.append(line[0])
  list_choices.append(line[2:])
  list_true_ind.append(int(line[1])) 


def toefl_func(words, vectors, vector_dimensions):
  """
  Computes accuracy for the task of selecting the semantically closest word to a target from a list of four candidates using cosine similarity

  Parameters
  ----------
  words : list
    A list containing only words (dtype = string) from given word vector file

  vectors : list
    A list of lists with each sub-list as vector (dtype = float) for corresponding word from given word vector file  

  vector_dimensions : int 
    The length of the vectors present in given word-vector file

  Returns
  -------
  arg1 : float
    accuracy score for the evaluation task
  """
  count = 0
  accuracy = 0
  missed_count = 0
  mean_vector = np.mean(vectors, axis=0)
  
  for question, choices, ans in tqdm(zip(list_questions,list_choices,list_true_ind)):
    mat = np.zeros((5,vector_dimensions))
    
    for i in range(5):
      mat[i,:] = mean_vector
    for word,vector in zip(words, vectors):
      if question == word:
        mat[0,:] = vector
      if choices[0] == word:
        mat[1,:] = vector
      if choices[1] == word:
        mat[2,:] = vector
      if choices[2] == word:
        mat[3,:] = vector
      if choices[3] == word:
        mat[4,:] = vector

    temp_bool = mat == 0
    if np.sum(temp_bool[1,:])==vector_dimensions:
      # print('Missed Word :', choices[0])
      missed_count+=1
      continue
    if np.sum(temp_bool[2,:])==vector_dimensions:
      # print('Missed Word :', choices[1])
      missed_count+=1
      continue
    if np.sum(temp_bool[3,:])==vector_dimensions:
      # print('Missed Word :', choices[2])
      missed_count+=1
      continue
    if np.sum(temp_bool[4,:])==vector_dimensions:
      # print('Missed Word :', choices[2])
      missed_count+=1
      continue

    a1 = mat[1:,:].reshape(4,vector_dimensions)
    a2 = mat[0,:].reshape(1,vector_dimensions)
    '''--------------------- FOR DEBUGGING ------------------------'''
    # print(question, choices[0], choices[1], choices[2], choices[3])
    # print(a1, a2)
    # print(np.shape(a1), np.shape(a2))
    # break
    ind = np.argmax(cosine_similarity(a1,a2))
    '''--------------------- FOR DEBUGGING ------------------------'''
    #print(question, choices)
    #print(ind, ans, cosine_similarity(a1,a2))
    #print('-'*25)

    if ind == ans:
      # print(question, choices, ans, cosine_similarity(a1,a2))
      count += 1    
  
  print('Missed Words/Questions :', missed_count)
  accuracy = count/len(list_questions)  

  return accuracy

In [0]:
#Global Context word vectors file
WordVec_path_a = 'Vector Representation of Words/3/3_word_vectors.txt'
#Skip Gram (SG) word vectors file
WordVec_path_b = 'Vector Representation of Words/2/GoogleNews-vectors-negative300.bin'
#Glove word vector file
WordVec_path_c = 'Vector Representation of Words/1/glove.6B.300d.txt'

#Global Context retrofitted word vectors file
outfile_path_a1 = 'Vector Representation of Words/3/RETRO_FrameNet_3_word_vectors.txt' 
outfile_path_a2 = 'Vector Representation of Words/3/RETRO_ppdb-xl_3_word_vectors.txt' 
outfile_path_a3 = 'Vector Representation of Words/3/RETRO_wordnet-synonyms_3_word_vectors.txt' 
outfile_path_a4 = 'Vector Representation of Words/3/RETRO_wordnet-synonyms+_3_word_vectors.txt' 

#Glove retrofitted word vector files
outfile_path_c1 = 'Vector Representation of Words/1/RETRO_FrameNet_glove.6B.300d.txt'
outfile_path_c2 = 'Vector Representation of Words/1/RETRO_ppdb-xl_glove.6B.300d.txt'
outfile_path_c3 = 'Vector Representation of Words/1/RETRO_wordnet-synonyms_glove.6B.300d.txt'
outfile_path_c4 = 'Vector Representation of Words/1/RETRO_wordnet-synonyms+_glove.6B.300d.txt'


In [0]:
'''Evaluating on the Glove and GC word vectors file (including retrofitted Glove and GC word vectors)'''

''' Read all the word vectors and normalize them '''
def read_word_vecs(filename,vector_dimensions):
  # Use this for Glove and Global Context Vectors
  words = []
  vectors = []
  fileObject = open(filename, 'r')
  for index,line in tqdm(enumerate(fileObject)):
    # if index%50000==0:print('->', index,'/ 3000000')
    line = line.strip().lower().split()
    words.append(line[0])
    # if len(line)-1 != vector_dimensions: continue
    vec = np.zeros(len(line)-1, dtype=float)
    if vec.shape[0] != vector_dimensions: 
      #print('Skipping at index :', index)
      continue
    for ind, vecVal in enumerate(line[1:]):
      vec[ind] = float(vecVal)
    ''' normalize weight vector '''
    vec /= math.sqrt((vec**2).sum() + 1e-6)
    vectors.append(vec)
    
  return words, vectors

''' change 2nd argument inside tuple to 50 or 300 accordingly for Global context (50) and Glove word vector files respectively '''   
filenames = [(outfile_path_a4,50), (outfile_path_c4,300)]  

for filename in filenames:
  words,vectors = read_word_vecs(filename[0],filename[1])
  print('word vec generated for :', filename)
  accuracy_score= toefl_func(words,vectors,vector_dimensions=filename[1])
  print('Accuracy :',accuracy_score)

In [0]:
'''Evaluating on the SG (skip gram) word vectors file (does not include retrofitted SG word vectors)'''

filename = WordVec_path_b 
def read_word_vectors_skipgram():
  # Use this for Google skipgram word2vec model
  word_vec_bin = KeyedVectors.load_word2vec_format(filename, binary=True)
  return word_vec_bin

## function to convert word_vec to (words, vectors)
def my_function(word_vec):
  words = []
  vectors = []
  vocabulary = word_vec.vocab
  for word in tqdm(vocabulary):
      words.append(word)  
      vectors.append(word_vec[word])
  
  return words, vectors    

word_vec_bin = read_word_vectors_skipgram()
words, vectors = my_function(word_vec_bin)
print('word vec generated for :', filename)
accuracy_score = toefl_func(words,vectors,300)
print('Accuracy :',accuracy_score)
